In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

color = sns.color_palette()
from plotnine import *

%matplotlib inline
#Packages used to display the Exploratory Data Analysis(EDA)
from pandas_profiling import ProfileReport
from IPython.display import IFrame

# Plotting interactive graphs
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
import string
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, TreebankWordTokenizer 
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
from sklearn.utils import resample
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.datasets import make_gaussian_quantiles
from sklearn.svm import SVC
from scipy import stats
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score,confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_predict, GridSearchCV, train_test_split

# Model imports
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier 

# Metrics
from sklearn.metrics import accuracy_score, log_loss, precision_score 
from sklearn.metrics import recall_score, precision_recall_curve, f1_score, classification_report, confusion_matrix
from collections import defaultdict

C:\Users\ADMIN\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train_df = pd.read_csv(r'C:/Users/ADMIN/Desktop/Everything/Hackathon/train_set.csv')
test_df = pd.read_csv(r'C:/Users/ADMIN/Desktop/Everything/Hackathon/test_set.csv')
sub_df = pd.read_csv(r'C:/Users/ADMIN/Desktop/Everything/Hackathon/sample_submission.csv')

In [3]:
train_df.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [4]:
trial = pd.DataFrame(train_df['lang_id'].value_counts()).reset_index()
trial.rename(
    columns={'index': 'language', 'lang_id': 'count'}, inplace=True)
trial

,language,count
0,tso,3000
1,ven,3000
2,sot,3000
3,afr,3000
4,eng,3000
5,zul,3000
6,nbl,3000
7,nso,3000
8,ssw,3000
9,xho,3000


In [5]:
lang_list =  [elements for elements in trial['language']]
unique_ls = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
lang_list

['tso', 'ven', 'sot', 'afr', 'eng', 'zul', 'nbl', 'nso', 'ssw', 'xho', 'tsn']

In [6]:
new_dict = zip(lang_list, unique_ls)
new_dict = dict(new_dict)
new_dict

{'tso': 1,
 'ven': 2,
 'sot': 3,
 'afr': 4,
 'eng': 5,
 'zul': 6,
 'nbl': 7,
 'nso': 8,
 'ssw': 9,
 'xho': 10,
 'tsn': 11}

In [7]:
train_df['id'] = [new_dict[elements]for elements in train_df['lang_id']]

In [8]:
train_df.head()

,lang_id,text,id
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10
2,eng,the province of kwazulu-natal department of tr...,5
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2


In [9]:
test_df.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [10]:
sub_df.head()

,index,lang_id
0,1,tsn
1,2,nbl


In [11]:
train_df['lang_id'].value_counts()

tso    3000
ven    3000
sot    3000
afr    3000
eng    3000
zul    3000
nbl    3000
nso    3000
ssw    3000
xho    3000
tsn    3000
Name: lang_id, dtype: int64

In [12]:
def remove_blanks(df):
    """
    Takes in a dataframe, detects empty strings and removes them.

    Parameters:
    ---------
    DataFrame 

    Returns:
    ---------
    DataFrame:Dataframe with no empty strings

    """
    blanks = []
    for index, tweet in enumerate(df['text']):
        if type(tweet) == str:
            if tweet in ['', ' ']:
                blanks.append(index)
    print(blanks)
    return df.drop(blanks)

In [13]:
train_df = remove_blanks(train_df)
test_df = remove_blanks(test_df)

[]
[]


In [14]:
def special_char_count(df):
    df["special_character_count"] = df["text"].apply(lambda character: sum(not character.isalpha() 
                                                                              for characters in character ))
    return(df)

In [15]:
special_char_count(train_df)

,lang_id,text,id,special_character_count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252
2,eng,the province of kwazulu-natal department of tr...,5,264
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239
...,...,...,...,...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...,11,272
32996,sot,modise mosadi na o ntse o sa utlwe hore thaban...,3,298
32997,eng,closing date for the submission of completed t...,5,222
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...,10,228


In [16]:
def word_count(df):
    df['word count'] = df['text'].apply(lambda t: len(t.split()))
    return(df)

In [17]:
word_count(train_df)

,lang_id,text,id,special_character_count,word count
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31
2,eng,the province of kwazulu-natal department of tr...,5,264,37
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43
...,...,...,...,...,...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...,11,272,49
32996,sot,modise mosadi na o ntse o sa utlwe hore thaban...,3,298,63
32997,eng,closing date for the submission of completed t...,5,222,37
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...,10,228,26


In [18]:
def remove_punctuation(text):
    return ''.join([words for words in text if words not in string.punctuation])

In [19]:
 def clean_message(df):
    df["clean_text"] = df["text"].apply(remove_punctuation)
    return(df)

In [20]:
clean_message(train_df)

,lang_id,text,id,special_character_count,word count,clean_text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24,umgaqosiseko wenza amalungiselelo kumaziko axh...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31,idha iya kuba nobulumko bokubeka umsebenzi nap...
2,eng,the province of kwazulu-natal department of tr...,5,264,37,the province of kwazulunatal department of tra...
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43,khomishini ya ndinganyiso ya mbeu yo ewa maana...
...,...,...,...,...,...,...
32995,tsn,popo ya dipolateforomo tse ke go tlisa boetele...,11,272,49,popo ya dipolateforomo tse ke go tlisa boetele...
32996,sot,modise mosadi na o ntse o sa utlwe hore thaban...,3,298,63,modise mosadi na o ntse o sa utlwe hore thaban...
32997,eng,closing date for the submission of completed t...,5,222,37,closing date for the submission of completed t...
32998,xho,nawuphina umntu ofunyenwe enetyala phantsi kwa...,10,228,26,nawuphina umntu ofunyenwe enetyala phantsi kwa...


In [21]:
def split_tweets(df):
    df["split_text"] = [tweets.split() for tweets in df["clean_text"]]
    return(df)

In [22]:
split_tweets(train_df).head()

,lang_id,text,id,special_character_count,word count,clean_text,split_text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24,umgaqosiseko wenza amalungiselelo kumaziko axh...,"[umgaqosiseko, wenza, amalungiselelo, kumaziko..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31,idha iya kuba nobulumko bokubeka umsebenzi nap...,"[idha, iya, kuba, nobulumko, bokubeka, umseben..."
2,eng,the province of kwazulu-natal department of tr...,5,264,37,the province of kwazulunatal department of tra...,"[the, province, of, kwazulunatal, department, ..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40,o netefatša gore o ba file dilo ka moka tše le...,"[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43,khomishini ya ndinganyiso ya mbeu yo ewa maana...,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


In [23]:
def text_stemmer(words, stemmer):
    return [stemmer.stem(word) for word in words]

In [24]:
def stemmed_text (df):
    df['stem'] = df["split_text"].apply(text_stemmer, args=(stemmer, ))
    return(df)

In [25]:
stemmed_text (train_df).head()

,lang_id,text,id,special_character_count,word count,clean_text,split_text,stem
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24,umgaqosiseko wenza amalungiselelo kumaziko axh...,"[umgaqosiseko, wenza, amalungiselelo, kumaziko...","[umgaqosiseko, wenza, amalungiselelo, kumaziko..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31,idha iya kuba nobulumko bokubeka umsebenzi nap...,"[idha, iya, kuba, nobulumko, bokubeka, umseben...","[idha, iya, kuba, nobulumko, bokubeka, umseben..."
2,eng,the province of kwazulu-natal department of tr...,5,264,37,the province of kwazulunatal department of tra...,"[the, province, of, kwazulunatal, department, ...","[the, provinc, of, kwazulunat, depart, of, tra..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40,o netefatša gore o ba file dilo ka moka tše le...,"[o, netefatša, gore, o, ba, file, dilo, ka, mo...","[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43,khomishini ya ndinganyiso ya mbeu yo ewa maana...,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...","[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


In [26]:
def df_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]

In [27]:
def word_lemma(df): 
    df['lemma'] = df['stem'].apply(df_lemma, args=(lemmatizer, )) 
    return(df)

In [28]:
word_lemma(train_df).head()

,lang_id,text,id,special_character_count,word count,clean_text,split_text,stem,lemma
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24,umgaqosiseko wenza amalungiselelo kumaziko axh...,"[umgaqosiseko, wenza, amalungiselelo, kumaziko...","[umgaqosiseko, wenza, amalungiselelo, kumaziko...","[umgaqosiseko, wenza, amalungiselelo, kumaziko..."
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31,idha iya kuba nobulumko bokubeka umsebenzi nap...,"[idha, iya, kuba, nobulumko, bokubeka, umseben...","[idha, iya, kuba, nobulumko, bokubeka, umseben...","[idha, iya, kuba, nobulumko, bokubeka, umseben..."
2,eng,the province of kwazulu-natal department of tr...,5,264,37,the province of kwazulunatal department of tra...,"[the, province, of, kwazulunatal, department, ...","[the, provinc, of, kwazulunat, depart, of, tra...","[the, provinc, of, kwazulunat, depart, of, tra..."
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40,o netefatša gore o ba file dilo ka moka tše le...,"[o, netefatša, gore, o, ba, file, dilo, ka, mo...","[o, netefatša, gore, o, ba, file, dilo, ka, mo...","[o, netefatša, gore, o, ba, file, dilo, ka, mo..."
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43,khomishini ya ndinganyiso ya mbeu yo ewa maana...,"[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...","[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...","[khomishini, ya, ndinganyiso, ya, mbeu, yo, ew..."


In [29]:
train_df.drop(["stem","lemma","split_text"],axis = 1).head()

,lang_id,text,id,special_character_count,word count,clean_text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...,10,220,24,umgaqosiseko wenza amalungiselelo kumaziko axh...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...,10,252,31,idha iya kuba nobulumko bokubeka umsebenzi nap...
2,eng,the province of kwazulu-natal department of tr...,5,264,37,the province of kwazulunatal department of tra...
3,nso,o netefatša gore o ba file dilo ka moka tše le...,8,217,40,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...,2,239,43,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [30]:
X = train_df['clean_text'].values
y = train_df['id'].values

In [31]:
vectorizer = TfidfVectorizer()
processed_features = vectorizer.fit_transform(X)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(processed_features, y, test_size=0.2, random_state=0)

In [33]:
text_classifier = RandomForestClassifier()
text_classifier.fit(X_train, y_train)

RandomForestClassifier()

In [34]:
predictions = text_classifier.predict(X_test)

In [35]:
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[597   0   0   0   0   0   0   0   0   0   0]
 [  0 599   0   0   0   0   0   0   0   0   0]
 [  0   0 600   0   0   0   0   0   0   0   0]
 [  0   0   0 587   0   0   0   0   0   0   0]
 [  0   0   0   0 574   0   0   0   0   0   0]
 [  0   0   0   1   3 615   5   0   2   8   0]
 [  0   0   0   0   1  19 585   0   0   8   0]
 [  0   0   0   0   0   0   0 568   0   0   2]
 [  0   0   0   1   0  15   0   0 608   0   0]
 [  0   0   0   1   0   6   2   0   0 612   0]
 [  0   0   0   0   0   0   0   0   0   0 581]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       597
           2       1.00      1.00      1.00       599
           3       1.00      1.00      1.00       600
           4       0.99      1.00      1.00       587
           5       0.99      1.00      1.00       574
           6       0.94      0.97      0.95       634
           7       0.99      0.95      0.97       613
           8       1.00      1.00      1.00      

In [36]:
from sklearn.metrics import mean_squared_error
RMSE  = np.sqrt(mean_squared_error(y_test, predictions))

In [37]:
RMSE

0.29669901083472294

In [38]:
clean_message(test_df)

,index,text,clean_text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele...",Mmasepala fa maemo a a kgethegileng a letlelel...
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.,Tshivhumbeo tshi fana na ngano dza vhathu
3,4,Kube inja nelikati betingevakala kutsi titsini...,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.,Winste op buitelandse valuta
...,...,...,...
5677,5678,You mark your ballot in private.,You mark your ballot in private
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ...",E Ka kopo etsa kgetho ya hao ka hloko hobane h...
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ...",TB ke bokudi ba PMB mme Morero o tla lefella t...


In [40]:
testing = test_df['clean_text'].values
tester = vectorizer.fit_transform(trial).toarray()

In [41]:
test_df.head()

,index,text,clean_text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele...",Mmasepala fa maemo a a kgethegileng a letlelel...
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.,Tshivhumbeo tshi fana na ngano dza vhathu
3,4,Kube inja nelikati betingevakala kutsi titsini...,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.,Winste op buitelandse valuta


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [43]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

In [44]:
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [45]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [46]:
from statsmodels.api import OLS
from geopy.distance import distance
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
import xgboost as xgb
from sklearn.tree import *
from sklearn.neural_network import *
from sklearn.neighbors import *
from sklearn.isotonic import *
from sklearn.gaussian_process import *
from sklearn import *
from sklearn.svm.classes import *
from sklearn.ensemble import*
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import add_constant
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR

C:\Users\ADMIN\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning:

The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.



In [47]:
clf = LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [48]:
vectorizer_dict = {'CV_1': CountVectorizer(max_df=0.8, min_df=3
                                           ),
                   'CV_2': CountVectorizer(ngram_range=(1, 3), max_df=0.8,
                                           min_df=3),
                   'CV_3': CountVectorizer(ngram_range=(2, 3), max_df=0.8,
                                           min_df=3),
                   'TF_1': TfidfVectorizer(max_df=0.8, min_df=3),
                   'TF_2': TfidfVectorizer(ngram_range=(1, 3), max_df=0.8,
                                           min_df=3),
                   'TF_3': TfidfVectorizer(ngram_range=(2, 3), max_df=0.8,
                                           min_df=3)}

In [49]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [50]:
model_dict = {
              'Non-linear SVM': SVC(gamma="scale"),
              'Random Forest': RandomForestClassifier(n_estimators = 1000),
                'Random Forest Reg': RandomForestRegressor(),
                'Support Vector Reg': SVR(),
              }

In [56]:
classifier_results_dict = defaultdict(list)
for vec_name, vectorizer in vectorizer_dict.items():

    X_train_v = vectorizer.fit_transform(X_train)
    X_test_v = vectorizer.transform(X_test)
    print(vec_name)  # keep track of progress

    for mod_name, model in model_dict.items():
        model.fit(X_train_v, y_train)
        y_pred_v = model.predict(X_test_v)

        RMSE  = np.sqrt(mean_squared_error(y_test, y_pred_v))

        classifier_results_dict['Vectorizer Type'].append(vec_name)
        classifier_results_dict['Model Name'].append(mod_name)
        classifier_results_dict[('RMSE')].append(RMSE)
        

classifier_results_df = pd.DataFrame(classifier_results_dict)

CV_1
CV_2
CV_3
TF_1
TF_2
TF_3


In [57]:
classifier_results_df = pd.DataFrame(classifier_results_dict)

In [58]:
classifier_results_df

,Vectorizer Type,Model Name,RMSE
0,CV_1,Non-linear SVM,0.378121
1,CV_1,Random Forest,0.510226
2,CV_1,Random Forest Reg,0.764579
3,CV_1,Support Vector Reg,0.972961
4,CV_2,Non-linear SVM,0.345906
5,CV_2,Random Forest,0.463547
6,CV_2,Random Forest Reg,0.764413
7,CV_2,Support Vector Reg,1.009881
8,CV_3,Non-linear SVM,1.932469
9,CV_3,Random Forest,1.767527


In [285]:
param_grid = {'alpha': [0.1, 1, 5, 10]}
text_clf = Pipeline([('tfid', TfidfVectorizer(min_df=1,
                                                max_df=0.8,
                                                ngram_range=(1, 4))),
                     ('grd',GridSearchCV(MultinomialNB(),
                                           param_grid=param_grid,
                                           cv=5,
                                           n_jobs=-1,
                                           scoring='f1_weighted'))])
text_clf.fit(X_train,y_train)

Pipeline(steps=[('tfid', TfidfVectorizer(max_df=0.8, ngram_range=(1, 4))),
                ('grd',
                 GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
                              param_grid={'alpha': [0.1, 1, 5, 10]},
                              scoring='f1_weighted'))])

In [286]:
predictions = text_clf.predict(testing)

In [287]:
my_predictions = text_clf.predict(X_test)

print("The RSME score is:",np.sqrt(mean_squared_error(y_test, my_predictions)))

The RSME score is: 0.08243315459839663


In [262]:
kaggle = pd.DataFrame()

In [263]:
kaggle['id'] = predictions
kaggle['index'] = np.arange(1, len(kaggle) + 1)

In [264]:
kaggle.head()

,id,index
0,11,1
1,7,2
2,2,3
3,9,4
4,4,5


In [265]:
kaggle['round'] = kaggle.id.round(1)
kaggle['round'] = kaggle['round'].apply(np.int64)
kaggle.head()

,id,index,round
0,11,1,11
1,7,2,7
2,2,3,2
3,9,4,9
4,4,5,4


In [266]:
key_list=list(new_dict.keys())
val_list=list(new_dict.values())
kaggle['lang_id'] = [key_list[val_list.index(value)] for value in kaggle['round']]

In [267]:
kaggle.drop(["round","id"],axis=1,inplace=True)

In [268]:
kaggle.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [269]:
kaggle.to_csv(r"Submission.csv",sep=',',index=False)